# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# File to Load:
file_to_load='../weatherpy/WeatherPy.csv' # csv file created in part I of homework 

# Read csv WeatherPy file and store into data frame:
WeatherPy_df=pd.read_csv(file_to_load)
WeatherPy_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,79.05,74,5.0,18.10,PF,1.649623e+09
1,Castro,-24.7911,-50.0119,67.80,72,34.0,3.91,BR,1.649623e+09
2,Camacha,33.0833,-16.3333,60.22,77,40.0,14.97,PT,1.649623e+09
3,Santa Cruz de la Sierra,-17.8000,-63.1667,87.85,58,40.0,23.02,BO,1.649623e+09
4,Departamento de Maldonado,-34.6667,-54.9167,78.22,64,7.0,16.08,UY,1.649623e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Determine max humidity level in the WeatherPy data frame:
max_hum=WeatherPy_df['Humidity'].max()


# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
WeatherPy_df = WeatherPy_df.dropna()
humidity = WeatherPy_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = WeatherPy_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=max_hum,
                               point_radius = 4)
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create new data frame with requested weather conditions:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows with null values

filtered_weather_df=WeatherPy_df[(WeatherPy_df['Max Temp'] < 80) &
                             (WeatherPy_df['Max Temp'] > 70) &
                             (WeatherPy_df['Wind Speed'] < 10) &
                             (WeatherPy_df['Cloudiness'] == 0)]

filtered_weather_df=filtered_weather_df.dropna() # drop any rows will null values

filtered_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
72,Arraial do Cabo,-22.9661,-42.0278,78.76,78,0.0,8.05,BR,1.649623e+09
294,Warner Robins,32.6210,-83.5999,76.93,24,0.0,9.22,US,1.649623e+09
339,Vallenar,-28.5708,-70.7581,75.51,31,0.0,7.63,CL,1.649623e+09
415,Avera,33.1940,-82.5271,74.55,32,0.0,6.98,US,1.649623e+09
518,Russell,32.3502,-85.1999,76.28,26,0.0,0.00,US,1.649623e+09
550,Bālurghāt,25.2167,88.7667,76.08,80,0.0,7.56,IN,1.649623e+09


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print('-------------------------------')
    


Retrieving Results for Index 72: Arraial do Cabo.
Closest hotel is Pousada Porto Praia.
-------------------------------
Retrieving Results for Index 294: Warner Robins.
Closest hotel is Super 8 by Wyndham Warner Robins.
-------------------------------
Retrieving Results for Index 339: Vallenar.
Closest hotel is Humacao Bed & Breakfast.
-------------------------------
Retrieving Results for Index 415: Avera.
Missing field/result... skipping.
-------------------------------
Retrieving Results for Index 518: Russell.
Closest hotel is Macs.
-------------------------------
Retrieving Results for Index 550: Bālurghāt.
Closest hotel is Circuit House, Balurghat.
-------------------------------


In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
72,Arraial do Cabo,-22.9661,-42.0278,78.76,78,0.0,8.05,BR,1.649623e+09,Pousada Porto Praia
294,Warner Robins,32.6210,-83.5999,76.93,24,0.0,9.22,US,1.649623e+09,Super 8 by Wyndham Warner Robins
339,Vallenar,-28.5708,-70.7581,75.51,31,0.0,7.63,CL,1.649623e+09,Humacao Bed & Breakfast
415,Avera,33.1940,-82.5271,74.55,32,0.0,6.98,US,1.649623e+09,NaN
518,Russell,32.3502,-85.1999,76.28,26,0.0,0.00,US,1.649623e+09,Macs
550,Bālurghāt,25.2167,88.7667,76.08,80,0.0,7.56,IN,1.649623e+09,"Circuit House, Balurghat"


In [8]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Customize the size of the figure:
# Add marker layer ontop of heat map
# Display figure

# Customize the size of the figure:
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Assign the marker layer to a variable:
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)


# Display figure
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig




Figure(layout=FigureLayout(height='420px'))